In [1]:
import os
import sys
project_root = os.path.abspath(os.path.join(os.getcwd(), '../../'))
sys.path.append(project_root)

In [10]:
import pandas as pd
from common.extractor.alpaca_extractor import AlpacaExtractor
from common.processor.processor import Processor

In [3]:
alpaca = AlpacaExtractor(paper=False)

In [24]:
orders = alpaca.orders()

In [25]:
orders = pd.DataFrame(orders)

In [26]:
orders

,id,client_order_id,created_at,updated_at,submitted_at,filled_at,expired_at,canceled_at,failed_at,replaced_at,...,stop_price,status,extended_hours,legs,trail_percent,trail_price,hwm,subtag,source,expires_at
0,a5c1e02f-c71f-4f69-b6d0-668b09c803fd,60c487fc-5453-4074-9a26-75696ad73ae6,2024-05-06T19:07:55.660781Z,2024-05-06T19:07:55.66614Z,2024-05-06T19:07:55.660781Z,2024-05-06T19:07:55.664426Z,None,None,None,None,...,None,filled,False,None,None,None,None,None,access_key,NaN
1,8b11eebb-fb4e-45e0-bbcb-11f2f176b8aa,fdf22764-3dbc-4f89-8802-554f563e1f0c,2024-05-10T19:06:53.911544Z,2024-05-10T19:06:53.916675Z,2024-05-10T19:06:53.911544Z,2024-05-10T19:06:53.914999Z,None,None,None,None,...,None,filled,False,None,None,None,None,None,access_key,NaN
2,362fb16c-857f-4cfd-97bc-2018c686a679,525247a0-58f4-49a2-9a2d-db5a06e4465d,2024-05-13T19:06:40.135267Z,2024-05-13T19:06:40.140107Z,2024-05-13T19:06:40.135267Z,2024-05-13T19:06:40.138736Z,None,None,None,None,...,None,filled,False,None,None,None,None,None,access_key,NaN
3,84bfec70-612f-4771-8122-04514837976f,893b24b7-bf45-4910-ac22-1cdb79e8edbc,2024-05-17T19:11:34.164599Z,2024-05-17T19:11:34.169301Z,2024-05-17T19:11:34.164599Z,2024-05-17T19:11:34.167719Z,None,None,None,None,...,None,filled,False,None,None,None,None,None,access_key,NaN
4,1a2bfb0e-5fc0-46bd-b590-f96e6ea2534e,528bfef5-531a-43bd-878c-1447eaf6c2dc,2024-05-20T19:07:20.57133Z,2024-05-20T19:07:20.576355Z,2024-05-20T19:07:20.57133Z,2024-05-20T19:07:20.574652Z,None,None,None,None,...,None,filled,False,None,None,None,None,None,access_key,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231,af3564ec-b697-472f-8a2c-8446fbdf8c73,e94d606c-f7f0-41eb-8e0c-b5f952c80759,2025-01-20T19:06:32.956071Z,2025-01-21T14:30:13.376197Z,2025-01-21T12:00:51.555973Z,2025-01-21T14:30:00.81Z,None,None,None,None,...,None,filled,False,None,None,None,None,None,access_key,2025-01-21T21:00:00Z
232,ff3b4492-0c71-4102-83d2-d4e137d5d473,6f665e4c-9844-4568-9058-e197a5b9169b,2025-01-24T19:05:46.697062Z,2025-01-24T19:05:46.715027Z,2025-01-24T19:05:46.7002Z,2025-01-24T19:05:46.707Z,None,None,None,None,...,None,filled,False,None,None,None,None,None,access_key,2025-01-24T21:00:00Z
233,81c0d966-7a21-46ac-a94c-5a074438aba8,da8b3720-5726-4a1c-84a7-352e26c2cbd4,2025-05-05T17:32:14.609611Z,2025-05-05T17:32:14.614882Z,2025-05-05T17:32:14.609611Z,2025-05-05T17:32:14.613949Z,None,None,None,None,...,None,filled,False,None,None,None,None,None,access_key,2025-05-05T20:00:00Z
234,87bce477-f1d0-402c-9e92-7c2ecd53d0a5,7f4be596-48be-4b0b-9e71-5867cba90c8b,2025-05-05T18:08:52.209491Z,2025-05-05T18:08:52.21443Z,2025-05-05T18:08:52.209491Z,2025-05-05T18:08:52.21335Z,None,None,None,None,...,None,filled,False,None,None,None,None,None,dashboard,2025-05-05T20:00:00Z


In [27]:
trades = orders[["filled_at","symbol","notional","filled_qty","filled_avg_price","side"]]
trades["date"] = [x.split("T")[0] for x in trades["filled_at"]]
trades = Processor.utc_date(trades)
trades["sell_date"] = trades["filled_at"].shift(-1)
trades["filled_avg_price"] = trades["filled_avg_price"].astype(float)
trades["filled_qty"] = trades["filled_qty"].astype(float)
trades["sell_avg_price"] = trades["filled_avg_price"].shift(-1)
trades = trades[trades["side"]=="buy"].dropna().drop("filled_at",axis=1)
trades["pnl"] = trades["sell_avg_price"] / trades["filled_avg_price"]

C:\Users\ericj\AppData\Local\Temp\ipykernel_24060\3828367461.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trades["date"] = [x.split("T")[0] for x in trades["filled_at"]]
C:\Users\ericj\Desktop\orivault\common\processor\processor.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["date"] = pd.to_datetime(data["date"], utc=True).dt.normalize()
C:\Users\ericj\AppData\Local\Temp\ipykernel_24060\3828367461.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFr

In [30]:
trades.sort_values("date",ascending=False).head(20)

,symbol,notional,filled_qty,filled_avg_price,side,date,sell_date,sell_avg_price,pnl
233,OI,5,0.373469,13.388000,buy,2025-05-05 00:00:00+00:00,2025-05-05T18:08:52.21335Z,13.37200,0.998805
231,MRNA,542.75,15.131029,35.870000,buy,2025-01-21 00:00:00+00:00,2025-01-24T19:05:46.707Z,41.41340,1.154541
229,MRNA,450,12.791359,35.180000,buy,2025-01-15 00:00:00+00:00,2025-01-17T19:06:03.712Z,34.08240,0.968800
228,MRNA,106.07,3.134054,33.843200,buy,2025-01-13 00:00:00+00:00,2025-01-15T14:30:01.11Z,35.18000,1.039500
226,CE,107.3,1.621924,66.156000,buy,2025-01-06 00:00:00+00:00,2025-01-10T19:05:57.587Z,65.40670,0.988674
224,ES,27.41,0.478393,57.296000,buy,2024-12-30 00:00:00+00:00,2025-01-06T14:30:01.103581Z,57.10000,0.996579
222,MKR/USD,28.7,0.018296,1537.890000,buy,2024-12-30 00:00:00+00:00,2024-12-30T16:42:33.130777Z,1475.22800,0.959255
220,CRV/USD,28.87,28.750416,0.984470,buy,2024-12-30 00:00:00+00:00,2024-12-30T01:45:06.535822Z,0.98380,0.999319
218,LINK/USD,29.13,1.320334,21.630000,buy,2024-12-30 00:00:00+00:00,2024-12-30T01:43:03.494763Z,21.55000,0.996301
216,BTC/USD,29.19,0.000306,93467.190000,buy,2024-12-30 00:00:00+00:00,2024-12-30T01:41:00.533151Z,93782.30000,1.003371
